# **Import Libraries**

In [1]:
pip install gradio nibabel tensorflow opencv-python numpy matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import cv2
import glob
import PIL
import shutil
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from skimage import data
from skimage.util import montage 
import skimage.transform as skTrans
from skimage.transform import rotate
from skimage.transform import resize
from PIL import Image, ImageOps  

# neural imaging
import nilearn as nl
import nibabel as nib
import nilearn.plotting as nlplt
!pip install git+https://github.com/miykael/gif_your_nifti # nifti to gif 
import gif_your_nifti.core as gif2nif

# ml libs
import keras
import keras.backend as K
from keras.callbacks import CSVLogger
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.layers import Rescaling, Normalization  # Corrected import

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

!pip3 install -U segmentation-models
%env SM_FRAMEWORK=tf.keras
import segmentation_models as sm
import tensorflow as tf
tf.keras.backend.set_image_data_format('channels_last')

#Code mới
import sys
import os
import glob
import random
import time

import numpy as np
import pandas as pd

import cv2
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from sklearn.model_selection import train_test_split
from segmentation_models import Unet, Linknet, PSPNet, FPN
import keras
from segmentation_models.utils import set_trainable
from torch.utils.data import Dataset
from keras.models import load_model
from tensorflow.keras import utils as np_utils


  Cloning https://github.com/miykael/gif_your_nifti to /tmp/pip-req-build-2f2wnpd1
  Running command git clone --filter=blob:none --quiet https://github.com/miykael/gif_your_nifti /tmp/pip-req-build-2f2wnpd1
  Resolved https://github.com/miykael/gif_your_nifti to commit 55c09c42921f4871cf43f63a303ab420a92a4f03
  Preparing metadata (setup.py) ... done
  Created wheel for gif_your_nifti: filename=gif_your_nifti-0.2.2-py3-none-any.whl size=6631 sha256=282033a59864de52c8821fcb0e2f795bca65c76a87b32c38fbde206635bc936f
  Stored in directory: /tmp/pip-ephem-wheel-cache-3mj93zl3/wheels/de/6d/35/5b931fbea2039fbd2ac937828c719c3cdd70953235af24b9ab
Successfully built gif_your_nifti


2025-06-02 06:40:04.628125: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748846404.860017      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748846404.924421      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.4 MB/s eta 0:00:00
env: SM_FRAMEWORK=tf.keras
Segmentation Models: using `tf.keras` framework.


APP ĐỌC HIỂN THỊ MÔ HÌNH 3D VÀ LÁT CẮT GIÚP QUAN SÁT TỔN THƯƠNG NHẤT ĐỊNH

In [3]:
import gradio as gr
import numpy as np
import nibabel as nib
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from skimage import measure

# Load model
model = tf.keras.models.load_model("unet_model.h5", compile=False)
IMG_SIZE = 128
PIXEL_SPACING_MM = 1  # giả định 1 pixel = 1mm

SEGMENT_CLASSES = {
    0: "Nền (Normal)",
    1: "Lõi khối u hoại tử (Core)",
    2: "Phù não (Edema)",
    3: "Khối u tăng sinh (Enhancing Tumor)"
}

def preprocess_volume(volume):
    # Đảm bảo shape (Slices, H, W)
    if volume.shape[2] < volume.shape[0]:
        volume = np.transpose(volume, (2, 0, 1))
    return volume

def resize_mask_to_orig(pred_mask, orig_shape):
    return cv2.resize(pred_mask.astype(np.uint8), (orig_shape[1], orig_shape[0]), interpolation=cv2.INTER_NEAREST)

def predict_slice(flair_slice, t1ce_slice):
    flair_resized = cv2.resize(flair_slice, (IMG_SIZE, IMG_SIZE))
    t1ce_resized = cv2.resize(t1ce_slice, (IMG_SIZE, IMG_SIZE))

    flair_norm = flair_resized / np.max(flair_resized) if np.max(flair_resized) > 0 else flair_resized
    t1ce_norm = t1ce_resized / np.max(t1ce_resized) if np.max(t1ce_resized) > 0 else t1ce_resized

    input_img = np.stack([flair_norm, t1ce_norm], axis=-1)[None, ...]  # shape (1, IMG_SIZE, IMG_SIZE, 2)

    pred = model.predict(input_img)[0]  # (IMG_SIZE, IMG_SIZE, 4)
    pred_mask = np.argmax(pred, axis=-1)
    return flair_resized, pred_mask

def predict_full_volume(flair, t1ce):
    flair = preprocess_volume(flair)
    t1ce = preprocess_volume(t1ce)
    n_slices = min(flair.shape[0], t1ce.shape[0])
    H, W = flair.shape[1], flair.shape[2]
    pred_volume = np.zeros((n_slices, H, W), dtype=np.uint8)
    for i in range(n_slices):
        _, pred_mask = predict_slice(flair[i], t1ce[i])
        pred_volume[i] = resize_mask_to_orig(pred_mask, (H, W))
    return pred_volume

def calculate_volumes(pred_volume):
    VOXEL_VOLUME_MM3 = PIXEL_SPACING_MM ** 3
    report = []
    for label in range(1, 4):
        voxel_count = np.sum(pred_volume == label)
        vol = voxel_count * VOXEL_VOLUME_MM3
        report.append(f"{SEGMENT_CLASSES[label]}: {vol:.1f} mm³")
    return "\n".join(report)

def plot_3d_volume(pred_volume, flair_volume, label):
    mask = (pred_volume == label)
    if np.sum(mask) == 0:
        fig = go.Figure()
        fig.add_trace(go.Scatter3d())  # empty figure
        return fig

    verts, faces, _, _ = measure.marching_cubes(mask.astype(float), level=0.5)
    brain_verts, brain_faces, _, _ = measure.marching_cubes(flair_volume, level=np.percentile(flair_volume, 20))

    brain_mesh = go.Mesh3d(
        x=brain_verts[:, 0], y=brain_verts[:, 1], z=brain_verts[:, 2],
        i=brain_faces[:, 0], j=brain_faces[:, 1], k=brain_faces[:, 2],
        color='lightgray', opacity=0.3, name='Não'
    )
    color_map = {1: "red", 2: "green", 3: "blue"}
    tumor_mesh = go.Mesh3d(
        x=verts[:, 0], y=verts[:, 1], z=verts[:, 2],
        i=faces[:, 0], j=faces[:, 1], k=faces[:, 2],
        color=color_map[label], opacity=0.6, name=SEGMENT_CLASSES[label]
    )
    fig = go.Figure(data=[brain_mesh, tumor_mesh])
    fig.update_layout(title=f"Mô hình 3D vùng {SEGMENT_CLASSES[label]} và não",
                      scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'),
                      margin=dict(r=10, l=10, b=10, t=30))
    return fig

def segment_and_visualize(flair_file, t1ce_file, slice_index, volume_label):
    if not flair_file or not t1ce_file:
        return None, "❌ Cần chọn cả 2 file flair.nii và t1ce.nii.", None, ""

    flair = nib.load(flair_file.name).get_fdata()
    t1ce = nib.load(t1ce_file.name).get_fdata()

    flair = preprocess_volume(flair)
    t1ce = preprocess_volume(t1ce)

    min_slices = min(flair.shape[0], t1ce.shape[0])
    if slice_index >= min_slices:
        return None, f"❌ Slice index {slice_index} vượt quá giới hạn (tối đa {min_slices - 1}).", None, ""

    # Dự đoán lát cắt
    flair_slice = flair[slice_index]
    flair_resized, pred_mask_2d = predict_slice(flair_slice, t1ce[slice_index])

    # Báo cáo lát cắt
    report_lines = []
    tumor_present = False
    for label in range(1, 4):
        mask = (pred_mask_2d == label).astype(np.uint8)
        pixel_count = np.sum(mask)
        if pixel_count > 0:
            tumor_present = True
            area_mm2 = pixel_count * (PIXEL_SPACING_MM ** 2)
            diameter_mm = np.sqrt(area_mm2 / np.pi) * 2
            coords = np.column_stack(np.where(mask))
            centroid = coords.mean(axis=0)
            report_lines.append(
                f"🩺 {SEGMENT_CLASSES[label]}:\n"
                f" - Diện tích xấp xỉ: {area_mm2:.1f} mm²\n"
                f" - Đường kính xấp xỉ: {diameter_mm:.1f} mm\n"
                f" - Vị trí trung tâm: (hàng={int(centroid[0])}, cột={int(centroid[1])})"
            )
    if not tumor_present:
        report_lines.append(f"✅ Không phát hiện khối u trong lát cắt {slice_index}.")

    # Vẽ 2D
    fig_2d, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(flair_resized, cmap='gray')
    ax[0].set_title("Flair Slice")
    ax[0].axis('off')

    ax[1].imshow(flair_resized, cmap='gray')
    ax[1].imshow(pred_mask_2d, cmap='jet', alpha=0.5)
    ax[1].set_title("Phân đoạn khối u")
    ax[1].axis('off')

    plt.tight_layout()

    # Dự đoán toàn volume và tính thể tích
    pred_volume = predict_full_volume(flair, t1ce)
    volumes_report = calculate_volumes(pred_volume)

    # Dựng 3D mô hình vùng tổn thương
    fig_3d = plot_3d_volume(pred_volume, flair, volume_label)

    return fig_2d, "\n\n".join(report_lines), fig_3d, volumes_report

with gr.Blocks() as demo:
    gr.Markdown("# Phân đoạn khối u não 3D")
    with gr.Row():
        flair_file = gr.File(label="Upload file Flair (.nii, .nii.gz)", file_types=[".nii", ".nii.gz"])
        t1ce_file = gr.File(label="Upload file T1ce (.nii, .nii.gz)", file_types=[".nii", ".nii.gz"])
    slice_index = gr.Slider(0, 150, value=60, step=1, label="Chọn lát cắt (Slice Index)")
    volume_label = gr.Radio(label="Chọn vùng khối u để hiển thị mô hình 3D", choices=[1, 2, 3], value=1)

    btn = gr.Button("Chạy phân đoạn")

    output_2d = gr.Plot(label="Kết quả phân đoạn 2D")
    output_report = gr.Textbox(label="Báo cáo lát cắt", lines=6)
    output_3d = gr.Plot(label="Mô hình 3D vùng tổn thương")
    output_volume_report = gr.Textbox(label="Thể tích khối u tổng thể (mm³)", lines=4)

    btn.click(
        fn=segment_and_visualize,
        inputs=[flair_file, t1ce_file, slice_index, volume_label],
        outputs=[output_2d, output_report, output_3d, output_volume_report]
    )

demo.launch()


I0000 00:00:1748846427.776079      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://e1b68f88a2031f2d94.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [4]:
pip install gradio nibabel numpy plotly scikit-image


Note: you may need to restart the kernel to use updated packages.


HIỂN THỊ MẶT CẮT VÀ PHÂN BỐ TỔN THƯƠNG CỦA NÃO

In [ ]:
import gradio as gr
import numpy as np
import nibabel as nib
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Load model
model = tf.keras.models.load_model("/kaggle/input/data-u-no/unet_model.h5", compile=False)
IMG_SIZE = 128
PIXEL_SPACING_MM = 1  # giả định mỗi pixel tương ứng 1mm x 1mm

SEGMENT_CLASSES = {
    0: "Nền (Normal)",
    1: "Lõi khối u hoại tử (Core)",
    2: "Phù não (Edema)",
    3: "Khối u tăng sinh (Enhancing Tumor)"
}

def segment_dual_modal(flair_file, t1ce_file, slice_index):
    if not flair_file or not t1ce_file:
        return None, "❌ Cần chọn cả 2 file flair.nii và t1ce.nii."

    try:
        flair = nib.load(flair_file.name).get_fdata()
        t1ce = nib.load(t1ce_file.name).get_fdata()
    except Exception as e:
        return None, f"❌ Lỗi đọc file MRI: {e}"

    # Đảm bảo shape (Slices, H, W)
    if flair.shape[2] < flair.shape[0]:
        flair = np.transpose(flair, (2, 0, 1))
    if t1ce.shape[2] < t1ce.shape[0]:
        t1ce = np.transpose(t1ce, (2, 0, 1))

    min_slices = min(flair.shape[0], t1ce.shape[0])
    if slice_index >= min_slices:
        return None, f"❌ Slice index {slice_index} vượt quá giới hạn (tối đa {min_slices - 1})."

    flair_slice = flair[slice_index]
    t1ce_slice = t1ce[slice_index]

    flair_resized = cv2.resize(flair_slice, (IMG_SIZE, IMG_SIZE))
    t1ce_resized = cv2.resize(t1ce_slice, (IMG_SIZE, IMG_SIZE))

    flair_norm = flair_resized / np.max(flair_resized) if np.max(flair_resized) > 0 else flair_resized
    t1ce_norm = t1ce_resized / np.max(t1ce_resized) if np.max(t1ce_resized) > 0 else t1ce_resized

    input_img = np.stack([flair_norm, t1ce_norm], axis=-1)[None, ...]

    pred = model.predict(input_img)[0]  # (IMG_SIZE, IMG_SIZE, 4)
    pred_mask = np.argmax(pred, axis=-1)

    report_lines = []
    tumor_present = False
    for label in range(1, 4):
        mask = (pred_mask == label).astype(np.uint8)
        pixel_count = np.sum(mask)
        if pixel_count > 0:
            tumor_present = True
            area_mm2 = pixel_count * (PIXEL_SPACING_MM ** 2)
            diameter_mm = np.sqrt(area_mm2 / np.pi) * 2
            coords = np.column_stack(np.where(mask))
            centroid = coords.mean(axis=0)
            report_lines.append(
                f"🩺 {SEGMENT_CLASSES[label]}:\n"
                f" - Diện tích xấp xỉ: {area_mm2:.1f} mm²\n"
                f" - Đường kính xấp xỉ: {diameter_mm:.1f} mm\n"
                f" - Vị trí trung tâm: (hàng={int(centroid[0])}, cột={int(centroid[1])})"
            )
    if not tumor_present:
        report_lines.append(f"✅ Không phát hiện bất kỳ khối u nào trong lát cắt {slice_index}.")

    fig, ax = plt.subplots(1, 2, figsize=(12, 5))
    ax[0].imshow(flair_resized, cmap='gray')
    ax[0].set_title("Flair Slice")
    ax[0].axis('off')

    ax[1].imshow(flair_resized, cmap='gray')

    # Hiển thị bản đồ phân đoạn với colormap jet
    im = ax[1].imshow(pred_mask, cmap='jet', alpha=0.5)
    ax[1].set_title("Phân đoạn khối u")
    ax[1].axis('off')

    # Tạo legend màu theo nhãn
    colors = plt.cm.jet(np.linspace(0, 1, len(SEGMENT_CLASSES)))
    patches = []
    for label, color in zip(SEGMENT_CLASSES.keys(), colors):
        patches.append(mpatches.Patch(color=color, label=SEGMENT_CLASSES[label]))

    ax[1].legend(handles=patches, loc='lower right', fontsize='small', framealpha=0.7)

    plt.tight_layout()
    plt.close(fig)

    return fig, "\n\n".join(report_lines)


gr.Interface(
    fn=segment_dual_modal,
    inputs=[
        gr.File(label="📂 Upload flair.nii", file_types=[".nii", ".nii.gz"]),
        gr.File(label="📂 Upload t1ce.nii", file_types=[".nii", ".nii.gz"]),
        gr.Slider(0, 150, value=60, step=1, label="Chọn lát cắt (Slice Index)"),
    ],
    outputs=[
        gr.Plot(label="🧠 Kết quả phân đoạn"),
        gr.Textbox(label="📋 Báo cáo chi tiết"),
    ],
    title="🧠 Brain Tumor Segmentation (Flair + T1CE)",
    description="Tải lên 2 file MRI: flair.nii và t1ce.nii. Mô hình sẽ phân đoạn khối u và in báo cáo kích thước, vị trí các vùng phù não, lõi u hoại tử và khối u tăng sinh."
).launch()

APP Hiển thị mô hình 3D vùng tổn thương khối u

In [5]:
# app_3d_volume.py
import gradio as gr
import numpy as np
import nibabel as nib
import tensorflow as tf
from skimage import measure
import plotly.graph_objects as go
import cv2

# Load model
model = tf.keras.models.load_model("/kaggle/input/data-u-no/unet_model.h5", compile=False)
IMG_SIZE = 128
PIXEL_SPACING_MM = 1

SEGMENT_CLASSES = {
    1: "Lõi khối u hoại tử (Core)",
    2: "Phù não (Edema)",
    3: "Khối u tăng sinh (Enhancing Tumor)"
}
COLOR_MAP = {1: "red", 2: "green", 3: "blue"}

def preprocess_volume(volume):
    if volume.shape[2] < volume.shape[0]:
        volume = np.transpose(volume, (2, 0, 1))
    return volume

def resize_mask_to_orig(pred_mask, orig_shape):
    return cv2.resize(pred_mask.astype(np.uint8), (orig_shape[1], orig_shape[0]), interpolation=cv2.INTER_NEAREST)

def predict_slice(flair_slice, t1ce_slice):
    flair_resized = cv2.resize(flair_slice, (IMG_SIZE, IMG_SIZE))
    t1ce_resized = cv2.resize(t1ce_slice, (IMG_SIZE, IMG_SIZE))
    flair_norm = flair_resized / np.max(flair_resized) if np.max(flair_resized) > 0 else flair_resized
    t1ce_norm = t1ce_resized / np.max(t1ce_resized) if np.max(t1ce_resized) > 0 else t1ce_resized
    input_img = np.stack([flair_norm, t1ce_norm], axis=-1)[None, ...]
    pred = model.predict(input_img)[0]
    pred_mask = np.argmax(pred, axis=-1)
    return resize_mask_to_orig(pred_mask, flair_slice.shape)

def predict_full_volume(flair, t1ce):
    n_slices = min(flair.shape[0], t1ce.shape[0])
    H, W = flair.shape[1:]
    pred_volume = np.zeros((n_slices, H, W), dtype=np.uint8)
    for i in range(n_slices):
        pred_volume[i] = predict_slice(flair[i], t1ce[i])
    return pred_volume

def plot_3d(pred_volume, flair, label):
    mask = (pred_volume == label)
    if np.sum(mask) == 0:
        return go.Figure()
    verts, faces, _, _ = measure.marching_cubes(mask.astype(float), level=0.5)
    brain_verts, brain_faces, _, _ = measure.marching_cubes(flair, level=np.percentile(flair, 20))

    return go.Figure(data=[
        go.Mesh3d(x=brain_verts[:, 0], y=brain_verts[:, 1], z=brain_verts[:, 2],
                  i=brain_faces[:, 0], j=brain_faces[:, 1], k=brain_faces[:, 2],
                  color='lightgray', opacity=0.3, name='Não'),
        go.Mesh3d(x=verts[:, 0], y=verts[:, 1], z=verts[:, 2],
                  i=faces[:, 0], j=faces[:, 1], k=faces[:, 2],
                  color=COLOR_MAP[label], opacity=0.6, name=SEGMENT_CLASSES[label])
    ])

def segment_3d(flair_file, t1ce_file, volume_label):
    if not flair_file or not t1ce_file:
        return go.Figure()

    flair = preprocess_volume(nib.load(flair_file.name).get_fdata())
    t1ce = preprocess_volume(nib.load(t1ce_file.name).get_fdata())

    pred_volume = predict_full_volume(flair, t1ce)
    fig = plot_3d(pred_volume, flair, volume_label)
    fig.update_layout(title=f"Khối u: {SEGMENT_CLASSES[volume_label]}")
    return fig

with gr.Blocks() as demo:
    gr.Markdown("## 🧠 Hiển thị mô hình 3D vùng tổn thương khối u")
    with gr.Row():
        flair_file = gr.File(label="Flair (.nii, .nii.gz)", file_types=[".nii", ".nii.gz"])
        t1ce_file = gr.File(label="T1ce (.nii, .nii.gz)", file_types=[".nii", ".nii.gz"])
    volume_label = gr.Radio(choices=[1, 2, 3], value=1, label="Chọn vùng khối u để dựng 3D")
    btn = gr.Button("Dựng mô hình 3D")
    output_plot = gr.Plot()

    btn.click(fn=segment_3d, inputs=[flair_file, t1ce_file, volume_label], outputs=output_plot)

demo.launch()


* Running on local URL:  http://127.0.0.1:7861
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://dda179b9891fd020b5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
